In [3]:
## install the pytorch3d package; hit 'shift+enter' keys to run
!conda create --name pytorch3d --yes
!conda activate pytorch3d
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch --yes
!conda install -c conda-forge -c takatosp1 fvcore --yes
!conda install pytorch3d -c pytorch3d --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/pytorch3d



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate pytorch3d
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/fastai

  added / updated specs:
    - fvcore


The following packages will be SUPERSEDED by a higher-priority channel:

In [4]:
## download the concerthall optimization code base; hit 'shift+enter' keys to run
!git clone https://github.com/alexacarlson/concerthall.git

Cloning into 'concerthall'...
remote: Enumerating objects: 521, done.
remote: Counting objects: 100% (521/521), done.
remote: Compressing objects: 100% (484/484), done.
remote: Total 521 (delta 72), reused 449 (delta 32), pack-reused 0
Receiving objects: 100% (521/521), 21.31 MiB | 12.73 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Checking connectivity... done.


In [7]:
## Train the graph convolutional neural network; hit 'shift+enter' keys to run
## NOTES on Training the Graph Convolutional Neural network to map 3D meshes --> audio quality parameters:
## To train your own graph convolutional network, you first need to create a folder in paperspace storage 
## at `/storage/concert_dataset` upload your data there. All of your meshes should be `.obj` files and
## should exist in a subdirectory of the dataset: `/storage/concert_dataset/OBJdatabase/`. 
## You will need to have a csv file of acoustic paramters (where each line is in the format `mesh.obj acoust_param1 acoust_param2 ... acoust_param10`) 
## located at and named `/storage/concert_dataset/AcousticParameters.csv`.
## The results (network weights and training loss values) will be saved to `/storage/mesh2acoustic_training_results`.
!cd concerthall && bash run_train_graphconv_regression.sh

RANDOM_SEED: 0
DATA:
  PATH: /storage/concert_dataset
BASE_MODEL:
  CLASSES: 10
  CONV_INIT: normal
  HIDDEN_DIMS: [128, 256, 512]
  INPUT_MESH_FEATS: 3
OPTIM:
  BATCH_SIZE: 4
  CLIP_GRADIENTS: 12.5
  EPOCH: 10
  LR: 0.0002
  MOMENTUM: 0.9
  VAL_BATCH_SIZE: 4
  WEIGHT_DECAY: 0.005
  WORKERS: 8
CHECKPOINT:
  best_acc: 0.0
  best_loss: 1.7976931348623157e+308
  experiment_path: /storage/mesh2acoustic_training_results/exp_03_23_19_41_34_concerthalloptim
  full_experiment_name: exp_03_23_19_41_34_concerthalloptim

config_yml          : config/mesh2audioparams_train.yml
config_override     : []
Splitting Dataset..
Training Samples: 24
Validation Samples: 3

 ***************** Training *****************
  0%|                                                    | 0/10 [00:00<?, ?it/s]Epoch: 0

	Training Loss: 35640240.166666664
 10%|████▍                                       | 1/10 [00:01<00:15,  1.72s/it]Epoch: 1

	Training Loss: 30029596.75
 20%|████████▊                                   |

In [8]:
## run the optimization framework; hit 'shift+enter' keys to run
## IMPORTANT INFO about running the acoustic optimization script:
## The Command Format for this script: 
## bash run_acousticoptim.sh INPUT_MESH SILHOUETTE_DEFORM_FLAG SILHOUETTE_REF_IMG ACOUSTIC_DEFORM_FLAG WHICH_ACOUSTIC_PARAMS OUTPUT_NAME
## where `INPUT_MESH` is the absolute filepath of the mesh you would like to deform, 
##       `SILHOUETTE_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for a specific 2D silhouette, 
##       `SILHOUETTE_REF_IMG` is the absolute filepath of the silhouette image you would like to use for mesh deformation, 
##       `ACOUSTIC_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for acostic quality, 
##       `WHICH_ACOUSTIC_PARAMS` is a string of the 10 acoustic quality parameters (each parameter needs to be separated by a comma with no spaces in between, see example below), 
##       `OUTPUT_NAME` is the absolute filepath where you would like the deformed output mesh to be saved.
##
## Example Command:
## bash run_acousticoptim.sh '/storage/quad_sphere.obj' True '/storage/square_sil.png' True '2660,24920,1.25,2066,17.7,57.7,0.54,0.31,9.37,1.55' 'test_sil_deform.obj'
##
## UNCOMMENT THIS LINE AND MODIFY INPUT PARAMS TO RUN
#!cd concerthall && bash run_acousticoptim.sh '/storage/quad_sphere.obj' True '/storage/square_sil.png' True '2660,24920,1.25,2066,17.7,57.7,0.54,0.31,9.37,1.55' 'test_sil_deform.obj'